In [1]:
import cv2
import numpy as np
from importnb import Notebook
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy.fft as fft
import numpy.matlib
from skimage.transform import rotate
import sys
from tqdm import tqdm_notebook as tqdm

In [2]:
with Notebook(): 
        import utils
        import pinkNoise

In [3]:
def blurLegend(directions, magnitude):
    norm = mpl.colors.Normalize(0, np.pi)
    plt.imshow(norm(directions))
    plt.show()

In [16]:
def makePatchKernel(img, r_min=5, r_max=31, inc_th=15):
    nr = r_max - r_min
    nt = int(180/inc_th)
    ind = []
    kernels = []
    ker_centre = (r_max*r_max)//2
#     kernels = np.zeros(nr*nt, r_max, img.shape[0]*img.shape[1])
    for r in range(r_min, r_max+1, 2):
        for theta in range(0, 180, inc_th):
            ind.append((r,theta))
            temp = np.zeros((r_max*r_max))
            ker = utils.blurKernel(r, theta).ravel()
#             print(temp[ker_centre-ker.shape[0]//2:ker_centre-ker.shape[0]//2].shape)
            temp[ker_centre-ker.shape[0]//2:ker_centre+ker.shape[0]//2+1] = ker
            kernels.append(temp)
#             print("\n\n", r, theta)
#             print(ker)
    return np.array(kernels), np.array(ind)

In [17]:
def generateKernels(r_max, inc_angle, patch_size, r_min=5):
    if r_min % 2==0:
        r_min += 1
        
    out = []
    for mag in range(r_max, r_max+1, 2):
        for ang in range(0, 180, inc_angle):
            kernel = utils.blurKernel(mag, ang)
            pd = (patch_size- kernel.shape[0])//2
            kernel = np.pad(kernel, ((pd, patch_size- kernel.shape[0]-pd), (pd, patch_size- kernel.shape[0]-pd)), 'constant', constant_values=(0, 0))
            out.append(kernel.ravel())
    
    print(np.array(out).shape)
    return np.array(out).T ##shape  patch_size x mag*ang

In [18]:
def getImageBlurEstimates(img, oimg, kernels, patch_size):
    pd = patch_size//2
    _,nimg = pinkNoise.addNoise(img, -1)
    nimg = np.pad(nimg, ((pd,pd), (pd,pd)), 'reflect')
    oimg = np.pad(img, ((pd,pd), (pd,pd)), 'reflect')
    
 
    cimg = utils.im2col(oimg, [patch_size, patch_size])
    cnimg = utils.im2col(nimg, [patch_size, patch_size])
    kidxs = []
    for k in tqdm(range(kernels.shape[1])):
        fi = np.sum(kernels[:,k][:,None] * cnimg, axis=0).reshape(img.shape)
        fi = np.pad(fi, ((pd,pd), (pd,pd)), 'reflect')
        cfi = utils.im2col(fi, [patch_size, patch_size])
        
        kidxs.append(np.sum((cfi - cimg)**2,axis=0))
        
    kidxs = np.argmin(np.array(kidxs), axis=0)
    
#     kidxs = np.argmin((cnimg.T @ kernels - cimg[patch_size**2//2, :][:,None])**2, axis = 1)
    
    del cnimg
    del cimg
    del nimg
    
    return kidxs

In [19]:
def findPatchKernel(img, r_min=5, r_max=31, inc_ang=15):
    mse = float('inf')
    opt_r, opt_theta = 1, 0
    if r_min % 2==0:
        r_min += 1
    if r_max % 2==0:
        r_max += 1
#     print(img.shape)
# #     kernels, ind = makePatchKernel(img, r_min, r_max, inc_ang)
#     pad_img = np.pad(img, ((r_max//2, r_max//2), (r_max//2, r_max//2)), 'constant', constant_values=(0, 0))
#     col_img = utils.im2col(pad_img, (r_max, r_max))
# #     out = kernels@col_img - img.ravel()
#     out = np.sum(np.power(kernels@col_img - img.ravel(), 2), axis=1)
#     print(out.shape)
    for r in range(r_min, r_max, 2):
        for theta in range(0, 180, inc_ang):
            blur = utils.linearBlur(img, r, theta)
            err = utils.MSELoss(img, blur)
            if err < mse:
                mse = err
        opt_r, opt_theta = r, theta

    return opt_r, opt_theta

In [20]:
def getImageBlurEstimate(img, patch_size = 31):
    blurField = np.zeros(img.shape)
    magnitude = np.zeros(img.shape)
    pd = patch_size // 2
    img = np.pad(img, ((pd, pd), (pd, pd)), 'reflect')
    from tqdm import tqdm_notebook as tqdm
    
    pd+=1
    for i in tqdm(range(pd, img.shape[0]-pd)):
        for j in range(pd, img.shape[1]-pd):
#             print(i-pd, j-pd, end = '\r')
#             print(img[i-pd:i+pd+1,j-pd:j+pd+1].shape)
            magnitude[i-pd,j-pd], blurField[i-pd,j-pd] = findPatchKernel(img[i-pd:i+pd+1,j-pd:j+pd+1])
#             magnitude[i-pd,j-pd], blurField[i-pd,j-pd] = findPatchKernel(img[i-pd:i+pd,j-pd:j+pd], 7, 8, 15)

    return magnitude, blurField

# TESTING

In [22]:
# # img = utils.loadImage('./resources/blu.png', 'gray')
# img = utils.loadImage('./resources/building_small.jpg', 'gray')
# img = np.array(img, dtype=np.float32)
# img = utils.linearBlur(img, 11, 45)
# # noise, noised_img = pinkNoise.addNoise(blurred_img, exponent = -1)
# kernels = generateKernels(14, 5, 35)
# sel_kernels = getImageBlurEstimates(img, img,kernels, 35)# _, plots = plt.subplots(1,2,figsize=(20,20))
# sel_kernels = getImageBlurEstimate(img, 35)# _, plots = plt.subplots(1,2,figsize=(20,20))

# print(sel_kernels.shape)
# # plots[0].imshow(noise, cmap = 'gray')
# # plots[1].imshow(noised_img, cmap = 'gray')
# # plt.show()
# print(sel_kernels[:,1500].reshape(21,21))


In [23]:
# print(np.unique(sel_kernels, return_counts=True))
# # print(np.sum(sel_kernes==114))
# kernels[:, sel_kernels][:,-1].reshape(21,21)

In [8]:
# magnitude, directions = getImageBlurEstimate(img[200:235,200:235])
# # magnitude, directions = findPatchKernel(img[100:100,200:350])
# print(magnitude, directions)

In [9]:
# blurLegend(directions, magnitude)
# np.set_printoptions(threshold=sys.maxsize)
# print((directions==45).sum()/(150*150))

In [19]:
A=np.random.uniform(size=(10, 9))
B=np.random.uniform(size=(9,100))

c = np.einsum('ij,jk->ijk', A,B)
np.argmin(np.sum(np.abs(c - B.reshape((1,9,100))), axis=1), axis=0)

array([1, 6, 1, 6, 6, 6, 6, 1, 6, 1, 8, 6, 1, 6, 8, 6, 1, 6, 6, 4, 1, 6,
       1, 6, 4, 1, 6, 1, 6, 4, 6, 6, 6, 6, 8, 1, 4, 6, 1, 6, 8, 1, 6, 6,
       4, 6, 6, 6, 6, 4, 6, 6, 1, 1, 6, 4, 1, 6, 4, 6, 6, 4, 8, 6, 1, 1,
       4, 6, 6, 4, 6, 4, 1, 4, 4, 6, 6, 6, 6, 4, 8, 1, 6, 1, 6, 6, 1, 4,
       8, 6, 6, 6, 1, 1, 8, 6, 6, 8, 6, 6])